# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
#!pip install datasets
# уже установлено

Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

import datetime as dt
import matplotlib.pyplot as plt
import re

import gc

In [3]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alexey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [6]:
# Загрузим датасет
dataset = load_dataset('imdb')

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [7]:
# Получить отдельные предложения и поместить их в sentences
sentences = []
dataset_text = dataset['train']['text'] + dataset['test']['text'] + dataset['unsupervised']['text']
dataset_text = dataset_text[:len(dataset_text)//10]

max_iter = len(dataset_text)
start = dt.datetime.now()
for i, sentence in enumerate(dataset_text):
    if i % 1000 == 0:
        print('complete: {} / {} after {}'.format(i, max_iter, dt.datetime.now()-start), end='\r')
    sentence = re.sub('(<br />)', ' ', sentence)
    sentence = re.sub('(<i>)|(</i>)|(<hr>)', '', sentence)
    sentence = re.sub(' +', ' ', sentence)
    sentences += [x.lower() for x in sent_tokenize(sentence)]
print('complete: {} / {} after {}'.format(max_iter, max_iter, dt.datetime.now()-start), end='\r')
print()

complete: 10000 / 10000 after 0:00:04.119887


In [8]:
print("Всего предложений:", len(sentences))

Всего предложений: 127921


Посчитаем для каждого слова его встречаемость.

In [9]:
# Расчет встречаемости слов
# - к нижнему регистру уже привели, повторно не приводим
# - знаки пунктуации не исключаем: хотим, чтобы они были и в генерируемом тексте
words = Counter()

max_iter = len(sentences)
start = dt.datetime.now()
for i, sentence in enumerate(sentences):
    if i % 5000 == 0:
        print('complete: {} / {} after {}'.format(i, max_iter, dt.datetime.now()-start), end='\r')
    words_sentence = word_tokenize(sentence)
    for word in words_sentence:
        words[word] += 1
print('complete: {} / {} after {}'.format(max_iter, max_iter, dt.datetime.now()-start), end='\r')
print()

complete: 127921 / 127921 after 0:00:16.123581


Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [10]:
# Наполнение словаря
vocab_size_target = 40000
vocab = set([w[0] for w in sorted(words.items(), key=lambda x: -x[1])][:vocab_size_target])
vocab |= set(['<unk>', '<bos>', '<eos>', '<pad>'])

In [11]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size_target + 4

In [12]:
print("Всего слов в словаре: {} / {}".format(len(vocab), len(words)))
print("Всего слов в тексте попало в словарь: {} / {}".format(
    sum([words[w] for w in vocab]), sum([words[w] for w in words])))
vocab_size = len(vocab)

Всего слов в словаре: 40004 / 62132
Всего слов в тексте попало в словарь: 2638905 / 2661037


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [13]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [14]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        tokenized_sentence = \
            [self.bos_id] + \
            [word2ind.get(word, self.unk_id) for word in word_tokenize(self.data[idx])] + \
            [self.eos_id]
        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [15]:
def collate_fn_with_padding(input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    max_seq_len = max([len(x) for x in input_batch])
    
    sequences = [sequence + [pad_id]*(max_seq_len-len(sequence)) for sequence in input_batch]
    sequences = torch.LongTensor(sequences).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

In [16]:
# оригинальный код, при котором train может и будет содержать часть объектов из eval и test
#train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
#eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

# исправленный код
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(eval_sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 32 # 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

In [17]:
print(len(train_dataset), len(eval_dataset), len(test_dataset))

102336 12792 12793


## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [18]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [19]:
losses_dict = {}
perplexities_dict = {}
model_parameters_cnt_dict = {}

In [20]:
def train_model(
    model,
    criterion,
    optimizer,
    train_dataloader,
    eval_dataloader,
    model_name,
    num_epoch,
):

    losses = []
    perplexities = []
    
    print('Training model: {}'.format(model_name))
    for epoch in range(num_epoch):
        print('Training epoch {}:'.format(epoch))
        epoch_losses = []
        model.train()
        start = dt.datetime.now()
        for i, batch in enumerate(train_dataloader):
            if i % 50 == 0:
                print('complete: {} after {}'.format(i, dt.datetime.now()-start), end='\r')
            optimizer.zero_grad()
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            loss.backward()
            optimizer.step()

            epoch_losses.append(float(loss.item()))
            
            torch.cuda.empty_cache()
            gc.collect()
        print('complete: {} after {}'.format(i, dt.datetime.now()-start), end='\r')
        print()

        losses.append(sum(epoch_losses) / len(epoch_losses))
        perplexities.append(evaluate(model, criterion, eval_dataloader))

    losses_dict[model_name] = losses
    perplexities_dict[model_name] = perplexities

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [21]:
class ModelSeminar(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int, num_layers: int, p_dropout: float):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)
        
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=p_dropout)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch) # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings) # [batch_size, seq_len, hidden_dim]
        output = self.non_lin(output)
        output = self.linear(output) # [batch_size, seq_len, hidden_dim]
        output = self.dropout(output)
        output = self.non_lin(output)
        projection = self.projection(output) # [batch_size, seq_len, vocab_size]

        return projection

In [22]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [23]:
# Обучаем модель
model_name = 'Seminar'
model = ModelSeminar(hidden_dim=256, vocab_size=vocab_size, num_layers=1, p_dropout=0.1).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, model_name, num_epoch=5)
model_parameters_cnt_dict[model_name] = int(sum(p.numel() for p in model.parameters())))

Training model: Seminar
Training epoch 0:
complete: 3197 after 0:11:04.839600
Training epoch 1:
complete: 3197 after 0:11:00.206586
Training epoch 2:
complete: 3197 after 0:11:00.560931
Training epoch 3:
complete: 3197 after 0:11:00.256561
Training epoch 4:
complete: 3197 after 0:11:01.758493


### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

Попробуем подобрать глубину RNN-слоя при сохранении числа параметров модели.

In [36]:
class ModelDepth(nn.Module):
    def __init__(self, hidden_dim: int, hidden_dim_rnn: int, vocab_size: int, num_layers: int, p_dropout: float):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim_rnn, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim_rnn, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)
        
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=p_dropout)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch) # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings) # [batch_size, seq_len, hidden_dim]
        output = self.non_lin(output)
        output = self.linear(output) # [batch_size, seq_len, hidden_dim]
        output = self.dropout(output)
        output = self.non_lin(output)
        projection = self.projection(output) # [batch_size, seq_len, vocab_size]

        return projection

In [38]:
# Обучаем модель
for num_layers in [2, 4, 8, 16]:
    model_name = 'Depth_'+str(num_layers)
    model = ModelDepth(hidden_dim=256, hidden_dim_rnn=256//num_layers, vocab_size=vocab_size, num_layers=num_layers, p_dropout=0.1).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    optimizer = torch.optim.Adam(model.parameters())

    train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, model_name, num_epoch=5)
    model_parameters_cnt_dict[model_name] = int(sum(p.numel() for p in model.parameters()))

Training model: Depth_2
Training epoch 0:
complete: 3197 after 0:11:25.646654
Training epoch 1:
complete: 3197 after 0:11:15.912732
Training epoch 2:
complete: 3197 after 0:11:12.750160
Training epoch 3:
complete: 3197 after 0:11:03.474826
Training epoch 4:
complete: 3197 after 0:11:00.553030
Training model: Depth_4
Training epoch 0:
complete: 3197 after 0:10:59.654489
Training epoch 1:
complete: 3197 after 0:10:59.393699
Training epoch 2:
complete: 3197 after 0:11:00.437878
Training epoch 3:
complete: 3197 after 0:10:59.451262
Training epoch 4:
complete: 3197 after 0:11:00.762286
Training model: Depth_8
Training epoch 0:
complete: 3197 after 0:11:01.607444
Training epoch 1:
complete: 3197 after 0:11:02.480984
Training epoch 2:
complete: 3197 after 0:11:00.985466
Training epoch 3:
complete: 3197 after 0:11:02.167462
Training epoch 4:
complete: 3197 after 0:11:01.637727
Training model: Depth_16
Training epoch 0:
complete: 3197 after 0:11:28.737136
Training epoch 1:
complete: 3197 after 

In [39]:
print(losses_dict)
print(perplexities_dict)
print(model_parameters_cnt_dict)

{'Seminar': [5.460784209676651, 4.9554973015418415, 4.750194182166314, 4.598084697282038, 4.476403498217193], 'Depth_2': [5.518228644650753, 5.039377156759218, 4.868351946032144, 4.727395237498614, 4.621624873085571], 'Depth_4': [5.652532223688953, 5.171000950033774, 5.001636923365924, 4.885779905945454, 4.802166089182574], 'Depth_8': [5.798401435365372, 5.326319493973084, 5.189257350096186, 5.102870675010037, 5.041971057858446], 'Depth_16': [6.046536235379905, 5.6409403023830125, 5.546018144650486, 5.491911851293672, 5.454644663174351]}
{'Seminar': [176.6740555381775, 169.07928508758545, 161.1207704925537, 161.51852466583253, 166.19407236099244], 'Depth_2': [186.34956680297853, 170.9232318687439, 166.47199472427369, 164.90686950683593, 170.74747200012206], 'Depth_4': [210.55562126159668, 185.13799417495727, 177.68036058425903, 175.13921827316284, 177.7818225479126], 'Depth_8': [237.36727066040038, 210.098755569458, 201.46299409866333, 198.63127086639403, 198.8620378303528], 'Depth_16'

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.

Лучше было бы построить графики, но из цифр это тоже видно: увеличение числа слоёв RNN-слоя при сохранении общего числа параметров модели (на самом деле, при небольшом его уменьшении) ухудшает качество модели. Других экспериментов я не проводил.